In [1]:
import pandas as pd
import numpy as np
import pysam
import os

In [3]:
sampleDictionary = {}
for file in os.listdir('/home/mark/Desktop/HLA/Globus/cleaned/'):
    if '.fasta.fai' in file:
        continue
    else:
        sample = file.split(".")[0]
        hap = file.split(".asm-")[1].split(".")[0]
        sampleDictionary[sample+"."+hap]='/home/mark/Desktop/HLA/Globus/cleaned/'+file
sampleDictionary

{'NA19836.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/NA19836.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00268.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/HG00268.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG03248.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/HG03248.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG02587.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/HG02587.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'NA19384.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/NA19384.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'NA19317.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/NA19317.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG02554.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/HG02554.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'NA19129.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/NA19129.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG00514.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/HG00514.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00731.hap2': '/home/mark/Desktop/HLA/Globus/cleaned

In [4]:
DRB_DF = pd.read_csv("/home/mark/Desktop/HLA/DRB/DRB_wPseudo_50percent.csv").drop(columns=['Unnamed: 0'])

In [5]:
drbGroup={}
for sample in set(DRB_DF['5']):
    tempDF = DRB_DF[DRB_DF['5']==sample].copy()
    group = '_'.join(set(sorted([x.split("-")[2].split("_")[0] for x in tempDF['10'] if 'HLA' in x])))
    drbGroup[sample]=group

In [6]:
drbGroup

{'HG03807.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'HG01505.hap2': 'DRB6_DRB9_DRB1',
 'HG03065.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'HG00513.hap2': 'DRB9_DRB1',
 'NA19705.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'HG03807.hap2': 'DRB6_DRB5_DRB9_DRB1',
 'NA12329.hap1': 'DRB4_DRB1_DRB7_DRB8_DRB9',
 'NA20509.hap1': 'DRB9_DRB1',
 'HG02666.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'HG04217.hap2': 'DRB2_DRB3_DRB9_DRB1',
 'HG03009.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'NA20355.hap2': 'DRB6_DRB9_DRB1',
 'HG00268.hap1': 'DRB6_DRB5_DRB9_DRB1',
 'HG03683.hap1': 'DRB4_DRB1_DRB7_DRB8_DRB9',
 'NA18989.hap1': 'DRB9_DRB1',
 'HG00171.hap2': 'DRB4_DRB1_DRB7_DRB8_DRB9',
 'HG03732.hap1': 'DRB4_DRB1_DRB7_DRB8_DRB9',
 'HG01457.hap2': 'DRB4_DRB1_DRB7_DRB8_DRB9',
 'HG03009.hap2': 'DRB2_DRB3_DRB9_DRB1',
 'HG00096.hap2': 'DRB2_DRB3_DRB9_DRB1',
 'HG02011.hap2': 'DRB6_DRB5_DRB9_DRB1',
 'HG03065.hap2': 'DRB2_DRB3_DRB9_DRB1',
 'NA19129.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'HG00268.hap2': 'DRB6_DRB5_DRB9_DRB1',
 'HG02818.hap1': 'DRB2_DRB3_DRB9_DRB1',
 'NA24385.hap2'

In [8]:
def pullSequence(genename):
    drb6_samples={}
    for assembly in set(DRB_DF['5']):
        drb6_samples[assembly]={}
        count = 1
        tempList=[]
        tempDF = DRB_DF[DRB_DF['5']==assembly].copy()
        for row in tempDF.index:
            if str(genename) in str(tempDF.at[row,'10']):
                tempList.append([x for x in tempDF.loc[row]])
            else:
                continue
        drb6_samples[assembly][count]=tempList

    myC4_Sequences={}
    for sample in drb6_samples.keys():

        for gene in drb6_samples[sample]:
            if len(drb6_samples[sample][gene])>0:

                geneList = drb6_samples[sample][gene]
                mygeneSeqs=[]
                start = int(geneList[0][6])
                end = int(geneList[-1][7])

                for exon in geneList:
                    exonStart = exon[6]
                    exonEnd = exon[7]
                    sequence = ''.join(pysam.faidx(sampleDictionary[sample], sample+":"+str(exonStart)+"-"+str(exonEnd)).split()[1:])
                    mygeneSeqs.append(sequence)

                myC4_Sequences[sample+":"+str(start)+"-"+str(end)+"_"+str(drbGroup[sample])]=''.join(mygeneSeqs)
            else:
                continue

    with open('/home/mark/Desktop/'+str(genename)+'_Sequences_Pulled.fasta', 'a+') as file:
        for key in myC4_Sequences.keys():
            file.write(">"+str(key)+"\n")
            file.write(myC4_Sequences[key]+"\n")
    file.close()

In [16]:
pullSequence('HLA-DRB9')